<center>Ángel Ríos San Nicolás</center>

# II Pasar a firma

Volvemos a construir el grupo de la curva elítpica y el punto de la misma del sistema de firma digital EC-DSA de Bitcoin.

In [1]:
p = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
E = EllipticCurve(GF(p), [0, 7])  # Curva elíptica y^2 = x ^ 3 + 7
N = E.cardinality()  # Número de puntos de la curva
Gx = int("79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798", base = 16) % p
Gy = int("483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8", base = 16) % p
G = E((Gx, Gy))

$\rightarrow$ Comprueba, utilizando Sage, que ambas firmas (la recién obtenida con el programa y la calculada con Sage)
son válidas para _mensaje_01.txt_. 
Consideramos el par de claves dadas en hexadecimal por 
$$\begin{array}{rl}
d = & 0x45737461206672617365206D65206C612067756172646F2070617261206DED2E\\
x = & 0x1693AAC9007A1C28B2DF4BEB207ECD6C6A150C324E2066897252756F3BFC375A\\
y = & 0x7F3EF84CC4BBD9C6988E3F77FE38127CF5B8C1513FC399CA26406774722DE01A\\
Q = & d\cdot G\end{array}$$

Y el mensaje dado también en hexadecimal por
$$\begin{array}{rcl}m & = & 546F646F20617175656C6C6F20636F727265206465206D69206375656E74612E \end{array}$$

In [2]:
d = int("0x45737461206672617365206D65206C612067756172646F2070617261206DED2E", base = 16) % p
x = int("0x1693AAC9007A1C28B2DF4BEB207ECD6C6A150C324E2066897252756F3BFC375A", base = 16) % p
y = int("0x7F3EF84CC4BBD9C6988E3F77FE38127CF5B8C1513FC399CA26406774722DE01A", base = 16) % p
Q = d * G
m = int("546F646F20617175656C6C6F20636F727265206465206D69206375656E74612E", base = 16) % p

Firmamos el mensaje utilizando el protocolo EC-DSA.

In [3]:
k = randint(2 * 255, 2 * 255 + 2 * 254)
xR = (k * G)[0]
r = ZZ(xR) % N; r ==0

False

In [4]:
F2 = GF(N)
s = ZZ(F2(k) ** (-1)) * (m + d * r) % N; s == 0

False

In [5]:
(hex(r),hex(s))

('0xf08b854640041fbfbfa8507d7a4e3c2e1555feef433d2a8e695428a282c3f9d6',
 '0x2badc06ed6c02e86135cc016e1fca4876b3190af3b0cce5ed28571213e6d3864')

Vamos a comprobar que la firma calculada es válida. Comprobamos que tomando $\tilde{s}\in\mathbb{Z}$ de modo que $s\cdot \tilde{s}=1\mod n$, la primera componente del punto $(m\tilde{s})G+(r\tilde{s})Q$ coincide, módulo $n$, con $r$.

In [6]:
r = int("13FEB3FA409C814EA0AC615C25AC8DE5DF1F2CE7BA192219F5782EB7AE6A25F6", base = 16)
s = int("14C25730AA71C116BD7600F2965C574A0B612A5F11F997EA8EEA4E3AE8690A6A", base = 16)
sinv = GF(N)(s)^(-1)
P = ZZ(m * sinv) * G + ZZ(r * sinv) * Q
print(ZZ(P[0]) % N == r % N)

True


$\rightarrow$ Realiza la verificación reemplazando $s$ por $n − s$. Demuestra que, si $(r, s)$ es una firma válida, también lo
es $(r, n − s)$.

Verificamos ahora que la firma también es válida reemplazando $s$ por $n-s$.

In [7]:
s = N - s
sinv = GF(N)(s)^(-1)
P = ZZ(m * sinv) * G + ZZ(r * sinv) * Q
print(ZZ(P[0]) % N == r % N)

True


Suponemos que $(r,s)$ es una firma válida y queremos probar que $(r,n-s)$ también es una firma válida.

Tomamos $\tilde{s},s'\in\mathbb{Z}$ tales que $s\tilde{s}=1\mod n$ y $(n-s)s'=1\mod n$. Claramente, se cumple $(n-s)s'=ns'-ss'=-ss'=s(-s')\mod n$ con lo que $s'=-\tilde{s}$. Tenemos entonces que

$$\left[\left((ms'\right)G+\left(rs'\right)Q\right]_0=\left[\left(m\left(-\tilde{s}\right)\right)G+(r\left(-\tilde{s}\right))Q\right]_0=\left[-\left(\left(m\tilde{s}\right)G+\left(r\tilde{s}\right)Q\right)\right]_0$$

donde $[A]_0$ denota la primera coordenada del punto $A$. Sabemos que el opuesto de un punto $A=[x:y:0]$ en el grupo $(E,\oplus)$ de la curva elíptica es $[x:-y:0]$ porque $[x:y:0]\oplus [x:-y:0]$ es el punto del infinito $[0:0:1]$, el elemento neutro, ya que la recta que pasa por $(x,y)$ y $(x,-y)$ es vertical. Por lo tanto, como solo cambia la segunda coordenada,
$$\left[-\left((m\tilde{s})G+(r\tilde{s})Q\right)\right]_0=\left[(m\tilde{s})G+(r\tilde{s})Q\right]_0=r\mod n$$
donde la última igualdad se tiene de que $(r,s)$ es una firma válida.

Por tanto, 
$$\left[\left(ms'\right)G+\left(rs'\right)Q\right]_0=r\mod n$$
y $(r,n-s)$ también es una firma válida.

$\rightarrow$ Con la clave privada correspondiente, hemos firmado _mensaje_01.txt_. ¿Se corresponde alguna de las
firmas siguientes con esa clave y este mensaje?
Tenemos la clave pública dada por
$$x = EA1901C0F7B9FB4F22F187CED808141EE30714ADEC3841A7C5209735E0E081E0$$
con $y$ impar. Recuperamos los cálculos de la hoja anterior donde calculamos la coordenada $y$.

In [8]:
# Tenemos la clave publica:
xq = int("EA1901C0F7B9FB4F22F187CED808141EE30714ADEC3841A7C5209735E0E081E0", base=16) % p
yq = sqrt(GF(p)(xq^3+7))
yq = ZZ(yq) # Como y es impar, es el que estábamos buscando.
Qq = E((xq,yq)) # Es un punto de la curva.
Qq.parent()

Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 7 over Finite Field of size 115792089237316195423570985008687907853269984665640564039457584007908834671663

Comprobamos que la __firma_03__ se corresponde con la clave pública.
$$\begin{array}{rl}
r = & 00D3FE590319899F970F2A5074CC931BE02EF5F1C702433FD611529389B4A20BA5\\
s = & 3315A17CFC631D3B9D2E22E739E795B2AC849C2638916071B2DB0EBCCA180C71\end{array}$$

In [9]:
r = int("00D3FE590319899F970F2A5074CC931BE02EF5F1C702433FD611529389B4A20BA5", base = 16)
s = int("3315A17CFC631D3B9D2E22E739E795B2AC849C2638916071B2DB0EBCCA180C71", base = 16)
sinv = GF(N)(s)^(-1)
P = ZZ(m * sinv) * G + ZZ(r * sinv) * Qq
print(ZZ(P[0]) % N == r % N)

True


Comprobamos que la __firma_04__ no se corresponde con la clave pública.
$$\begin{array}{rl}
r = & 27871E5EDFF45363961D5FA96E9C8219FFAEDCF50BD06B663A4DC97B04506324\\
s = & 6EB2BAC70ED0F499FF78CA59721EC469AA89C106B4540731ACAEBB6ABD370361
\end{array}$$

In [10]:
# Comprobamos si firma_04 se corresponde con la clave pública
r = int("27871E5EDFF45363961D5FA96E9C8219FFAEDCF50BD06B663A4DC97B04506324", base = 16)
s = int("6EB2BAC70ED0F499FF78CA59721EC469AA89C106B4540731ACAEBB6ABD370361", base = 16)
sinv = GF(N)(s)^(-1)
P = ZZ(m * sinv) * G + ZZ(r * sinv) * Qq
print(ZZ(P[0]) % N == r % N)

False


Comprobamos que la __firma_05__ se corresponde con la clave pública.
$$\begin{array}{rl}
r = & 3B99865AD8A13A74E08512553E7423DB164151905EB5C657A567F02A89277DE8\\
s = & 68E83F426286CC2C266EA6431262E21F65B010430E0962DF40B0461493822507\end{array}$$

In [11]:
# Comprobamos si firma_05 se corresponde con la clave pública
r = int("3B99865AD8A13A74E08512553E7423DB164151905EB5C657A567F02A89277DE8", base = 16)
s = int("68E83F426286CC2C266EA6431262E21F65B010430E0962DF40B0461493822507", base = 16)
sinv = GF(N)(s)^(-1)
P = ZZ(m * sinv) * G + ZZ(r * sinv) * Qq
print(ZZ(P[0]) % N == r % N)

True


$\rightarrow$ En realidad, las tres son firmas reales para la clave _publica_02.raw_. Lo que ocurre es que no todas firman
el mensaje _mensaje_01.txt_. ¿Sabrías recuperar ese o esos otros mensajes?

Para recuperar el mensaje $m$, tenemos la ecuación

$$[(m\tilde{s})G + (r\tilde{s})Q]_0= r\qquad\text{en }\mathbb{F}_{n} $$

donde $[A]_0$ denota la primera componente de un punto $A\in E$ y $\tilde{s}=s^{-1}$ en $\mathbb{F}_n$. Sabemos que a partir de la primera componente podemos calcular la segunda resolviendo $y^2= x^3+ax+b$ en $\mathbb{F}_n$ con lo que podemos calcular $P\in  E$ tal que

$$m\tilde{s}G+r\tilde{s}Q = P.$$

Multiplicando por $s$,

$$mG= sP-rQ\in E.$$

Calcular $m$ a partir de esta información se corresponde a calcular el logaritmo discreto de $sP-rQ$ en base G en el grupo $\left(E,\oplus\right)$ de la cuva elíptica. Este es un problema como el de la mochila, muy difícil de resolver computacionalmente que no somos capaces de calcular eficientemente en la práctica.

Aunque conozcamos la clave privada $d$ tal que $Q = d\cdot G$,

$$m\tilde{s}G+r\tilde{s}dG=P.$$

Multiplicando por $s$,

$$mG+rdG=sP\longrightarrow (m+rd)G=sP$$

y el problema consiste nuevamente en calcular el logaritmo discreto de $sP$ en base $G$. 

## Firma de transacciones

$\rightarrow$ Para su firma, el hash resaltado en gris ha de interpretarse como un número escrito con la codificación **bigendian**
(h = 0x169E...). ¿Constituye una firma válida para ese mensaje con respecto a la clave pública
marcada en verde el par destacado en amarillo?
Comprobamos si la siguiente firma $(r,s)$ es válida para el mensaje $t$ con la clave pública $(x,y)$.

In [12]:
r = int("4E45E16932B8AF514961A1D3A1A25FDF3F4F7732E9D624C6C61548AB5FB8CD41", base = 16)
s = int("181522EC8ECA07DE4860A4ACDD12909D831CC56CBBAC4622082221A8768D1D09", base = 16)
x = int("11DB93E1DCDB8A016B49840F8C53BC1EB68A382E97B1482ECAD7B148A6909A5C", base = 16)
y = int("B2E0EADDFB84CCF9744464F82E160BFA9B8B64F9D4C03F999B8643F656B412A3", base = 16)
t = int("169E1E83E930853391BC6F35F605C6754CFEAD57CF8387639D3B4096C54F18F4", base = 16)
Q2 = E((x,y))
sinv = GF(N)(s)^(-1)
P = ZZ(t * sinv) * G + ZZ(r * sinv) * Q2
print(ZZ(P[0]) % N == r % N)

False


Como $\left[(t\tilde{s})G+(r\tilde{s})Q\right]_0\neq r\mod n$, la firma no es válida.


$\rightarrow$ El ejecutable _02_prepara_tr_ devuelve el hash de la transacción modificada. Comprueba que (esta vez sí)
la firma incluida en la segunda transacción del bloque #170 es válida.
Comprobamos ahora que la firma es válida para la transacción modificada $t'$.

In [13]:
t2 = int("7A05C6145F10101E9D6325494245ADF1297D80F8F38D4D576D57CDBA220BCB19", base = 16)
P = ZZ(t2 * sinv) * G + ZZ(r * sinv) * Q2
print(ZZ(P[0]) % N == r % N)

True


Esta vez $\left[(t'\tilde{s})G+(r\tilde{s})Q\right]_0 = r\mod n$ y la firma sí es válida.